# RAG_toy

**Over 'FinanceBench' tiny database**:

-[MICROSOFT_2023_10K](https://github.com/castillosebastian/genai0/blob/main/data/financebench/MICROSOFT_2023_10K.pdf)    
-[JOHNSON&JOHNSON_2022Q4_EARNINGS](https://github.com/castillosebastian/genai0/blob/main/data/financebench/JOHNSON&JOHNSON_2022Q4_EARNINGS.pdf)   
-[Pfizer_2023Q2_10Q](https://github.com/castillosebastian/genai0/blob/main/data/financebench/Pfizer_2023Q2_10Q.pdf)   
-[BESTBUY_2017_10K](https://github.com/castillosebastian/genai0/blob/main/data/financebench/BESTBUY_2017_10K.pdf)   
-[BESTBUY_2019_10K](https://github.com/castillosebastian/genai0/blob/main/data/financebench/BESTBUY_2019_10K.pdf)   

# 1. Set credential to AI-Search and Azure OpenAI (keep them secret)

In [1]:
#!pip install azure-search-documents-11.4.0b8

In [1]:
#! pip install openai langchain --quiet
# keys (keep private)----------------------------------------------
import os
from azure.core.credentials import AzureKeyCredential
os.environ["AZURE_OPENAI_API_KEY"]  = 'b82effcf491e45a088b1cd578713311c'
os.environ["OPENAI_API_VERSION"]    = '2023-05-15'
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://usesharedaopenai001.openai.azure.com/'
os.environ["OPENAI_API_TYPE"]       = 'azure'
# Variables-------------------------------------------------------
index                               = "azure-cognitive-search-vector-demo"
azure_search_endpoint               = 'https://genai0.search.windows.net'
#MODEL                               = "gtp35turbo-latest"
MODEL                               = 'gpt-3.5-turbo'
key                                 = 'lvhCA67EeE3JRyxyem5L0wGJSfOxscm2jft887ECdJAzSeDzoCNZ'
model                               = "text-embedding-ada-002"
credential                          = AzureKeyCredential(key)
COMPLETION_TOKENS                   = 1000
top_search_vector_k                 = 5

# 2. Some helper functions

In [2]:
import os
import sys
import pandas as pd
import openai
import json
import openai
from openai import AzureOpenAI
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.docstore.document import Document
#from azure.search.documents.models import VectorizedQuery
from langchain.vectorstores.azuresearch import AzureSearch
from IPython.display import display, HTML, Markdown
# Some helper functions---------------------------------------------
# Retriever------------------------------------------------------------
from azure.search.documents import SearchClient
search_client = SearchClient(endpoint=azure_search_endpoint, index_name=index, credential=credential)

In [3]:
# Import required libraries
import os
from dotenv import load_dotenv
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from llama_index.vector_stores.cogsearch import (
    IndexManagement,
    CognitiveSearchVectorStore,
)
from langchain.embeddings import OpenAIEmbeddings
from llama_index.llms import AzureOpenAI
from llama_index import (        
    StorageContext,
    ServiceContext,
    VectorStoreIndex,
)

In [4]:
metadata_fields = {
    "company_name":"company_name",
    "source": "source",
    "doc_type": "doc_type"
}

vector_store = CognitiveSearchVectorStore(
    search_or_index_client=search_client,
    filterable_metadata_field_keys=metadata_fields,
    index_management=IndexManagement.NO_VALIDATION,
    id_field_key="id",
    chunk_field_key="page_content",
    embedding_field_key="contentVector",  
    metadata_string_field_key="company_name",  
    doc_id_field_key="source",  
)

In [5]:
# define embedding function
from llama_index.embeddings import OpenAIEmbedding
from llama_index import (
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    VectorStoreIndex,
)

llm = AzureChatOpenAI(model_name=MODEL)
embed_model = AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002",chunk_size=1000)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(embed_model=embed_model)
index = VectorStoreIndex.from_documents(
    [], storage_context=storage_context, service_context=service_context
)

In [6]:
query_engine = index.as_query_engine()
response = query_engine.query("What was a hard moment for the author?")
display(Markdown(f"<b>{response}</b>"))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [5]:
index_client = SearchIndexClient(
    endpoint=azure_search_endpoint,
    credential=credential,
)

In [6]:
metadata_fields = {
    "company_name":"company_name",
    "source": "source",
    "doc_type": "doc_type"
}

In [7]:
vector_store = CognitiveSearchVectorStore(  
    search_or_index_client=index_client,  
    index_name=index,  
    filterable_metadata_field_keys = metadata_fields,  
     index_management=IndexManagement.NO_VALIDATION,
    id_field_key="id",  
    chunk_field_key="page_content",  
    embedding_field_key="contentVector",  
    metadata_string_field_key="company_name",  
    doc_id_field_key="source",  
)  

In [8]:
#from langchain.chat_models import AzureChatOpenAI
llm = AzureChatOpenAI(model_name=MODEL)
embed_model = AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002",chunk_size=1000)
 

In [9]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)


https://docs.llamaindex.ai/en/stable/examples/vector_stores/CognitiveSearchIndexDemo.html

In [10]:
index = VectorStoreIndex.from_documents(
    [], storage_context=storage_context, service_context=service_context
)

In [12]:
query_engine = index.as_query_engine()
response = query_engine.query("What is the Revenue of Microsoft")
print(response)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# 4. Retrieve-Augment-Generate

In [4]:
vector_query = VectorizedQuery(vector=generate_embeddings(QUESTION), k_nearest_neighbors=5, fields="contentVector")
# Using hybrid Search
results = search_client.search(
    search_text=QUESTION, # arroja error the tokens
    #search_text=None,
    vector_queries= [vector_query],
    include_total_count=True,
    select=["id", "company_name", "source", "doc_type", "page_content"],
    top=5
)

retrieved_docs = False

if retrieved_docs:
    #results.get_count()
    for result in results:  
        print(f"Company_name: {result['company_name']}")  
        print(f"Score: {result['@search.score']}")  
        print(f"Source: {result['source']}")  
        print(f"Doc_type: {result['doc_type']}")  
        print(f"Page_content: {result['page_content']}\n")
        print('-'*100)  

# Format Azure AI Search results as an ordered dictionary
from collections import OrderedDict
ordered_results = OrderedDict()
for result in results:
    ordered_results[result['id']]={
        "score": result['@search.score'],
        "company_name": result['company_name'],
        "source": result['source'],
        "doc_type": result['doc_type'],
        "page_content": result['page_content']
    }
# From the ordered dictionary, build the "Documents" array, needed by langchain "qa_with_sources" chain
top_docs = []
for key,value in ordered_results.items():
    references = f'Company: {value["company_name"]}, SEC report: {value["doc_type"]} \n (reference doc:"{value["source"]}", reference_score:{value["score"]} - id:{key}).'
    top_docs.append(Document(page_content=value["page_content"], metadata={"source": references}))

In [5]:
# Prepare the Open AI deployment
#
if final_answer:    
    from langchain.chat_models import AzureChatOpenAI
    llm = AzureChatOpenAI(model_name=MODEL)
    from langchain.chains.qa_with_sources import load_qa_with_sources_chain
    chain = load_qa_with_sources_chain(llm, chain_type='stuff')
    response = chain({"input_documents": top_docs, "question": QUESTION, "language": "English"})

# 5. Answer

In [6]:
if final_answer:    
    display(HTML(f"<br/><br/><b>RAG_toybot final answer:</b>"))
    display(Markdown(response['output_text']))
else:
    for i, doc in enumerate(top_docs):  
        print(f"Retrieved document {i}\n\nContent: {doc.page_content}")
        print(f"Retrieved document {i}\n\nMetadata: {doc.metadata}")
        print('-'*100)  
    print(top_docs)

The revenue of JOHNSON&JOHNSON for the full year 2022 was $94,943 million.
SOURCES: Company: JOHNSON&JOHNSON, SEC report: EARNINGS (reference doc:"../../data/financebench/JOHNSON&JOHNSON_2022Q4_EARNINGS.pdf")